# *Dannie Vo , Nancy Tran*
## Project 1: Fall Detection Data from China
## Subtitle: Fall Detection

### 1. Objective of the Proposed Research:

### 2. Data Exploration:

In [1]:
mydata <- read.csv('~/Documents/ANLT212/falldetection.csv', header = TRUE, stringsAsFactors = FALSE)
head(mydata,10)

ACTIVITY,TIME,SL,EEG,BP,HR,CIRCLUATION
<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
3,4722.92,4019.64,-1600.00,13,79,317
2,4059.12,2191.03,-1146.08,20,54,165
2,4773.56,2787.99,-1263.38,46,67,224
4,8271.27,9545.98,-2848.93,26,138,554
4,7102.16,14148.80,-2381.15,85,120,809
5,7015.24,7336.79,-1699.80,22,95,427
3,8620.28,24949.90,-3198.06,35,157,1519
3,9238.73,39245.50,-2590.00,15,196,1885
0,12276.40,59742.00,-5101.00,56,249,2826


In [2]:
cat('The fall detection dataset has', dim(mydata)[1],'rows and',dim(mydata)[2],'columns.')

The fall detection dataset has 16382 rows and 7 columns.

In [3]:
suppressMessages(library(dplyr))
mydata %>% count(ACTIVITY) %>% arrange(desc(n))

ACTIVITY,n
<int>,<int>
0,4608
3,3588
4,3494
2,2502
5,1688
1,502


Based on the activity level with 0: Standing, 1: Walking, 2: Sitting, 3: Falling, 4: Cramps, 5 Running, we can see that the largest amount of fall datset was recorded during the time when the old patient is standing, followed by when the patient is falling.

Below is the summary of the data, showing the 25th and 75th quantile as well as min/median/mean/max of each column:

In [4]:
summary(mydata)

    ACTIVITY          TIME             SL                 EEG           
 Min.   :0.000   Min.   : 1954   Min.   :     42.2   Min.   :-12626000  
 1st Qu.:0.000   1st Qu.: 7264   1st Qu.:   9941.2   1st Qu.:    -5630  
 Median :3.000   Median : 9769   Median :  31189.2   Median :    -3361  
 Mean   :2.361   Mean   :10937   Mean   :  75272.0   Mean   :    -5621  
 3rd Qu.:4.000   3rd Qu.:13482   3rd Qu.:  80761.4   3rd Qu.:    -2150  
 Max.   :5.000   Max.   :50896   Max.   :2426140.0   Max.   :  1410000  
       BP               HR         CIRCLUATION   
 Min.   :  0.00   Min.   : 33.0   Min.   :    5  
 1st Qu.: 25.00   1st Qu.:119.0   1st Qu.:  587  
 Median : 44.00   Median :180.0   Median : 1581  
 Mean   : 58.25   Mean   :211.5   Mean   : 2894  
 3rd Qu.: 78.00   3rd Qu.:271.0   3rd Qu.: 3539  
 Max.   :533.00   Max.   :986.0   Max.   :52210  

Below shows the data type of each column in the dataset:

In [5]:
str(mydata)

'data.frame':	16382 obs. of  7 variables:
 $ ACTIVITY   : int  3 2 2 4 4 5 3 3 0 4 ...
 $ TIME       : num  4723 4059 4774 8271 7102 ...
 $ SL         : num  4020 2191 2788 9546 14149 ...
 $ EEG        : num  -1600 -1146 -1263 -2849 -2381 ...
 $ BP         : int  13 20 46 26 85 22 35 15 56 82 ...
 $ HR         : int  79 54 67 138 120 95 157 196 249 315 ...
 $ CIRCLUATION: int  317 165 224 554 809 427 1519 1885 2826 5844 ...


In [6]:
mydata[mydata$HR %in% c(60:100),] %>% count(ACTIVITY) %>% arrange(desc(n))

ACTIVITY,n
<int>,<int>
2,617
3,415
0,359
4,285
1,202
5,142


As the normal heart rate is between 60 and 100, I noticed that most of old patients that have normal heart rate recorded at the time are sitting.

In [7]:
cat('The first few cases where patients had high blood pressure:\n')
head(mydata[mydata$BP > 180,])
cat('There are',nrow(mydata[mydata$BP > 180,]),'cases where the old patients had high blood pressure with their systolic number > 180')

The first few cases where patients had high blood pressure:


,ACTIVITY,TIME,SL,EEG,BP,HR,CIRCLUATION
,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
29,0,23099.1,280276,-13215,264,465,8741
43,0,20309.9,291719,-11688,242,487,10978
145,0,13372.0,190854,-8983,252,350,7945
161,0,21183.2,324568,-13759,323,482,10983
230,0,15496.8,128042,-8414,251,325,5739
310,0,16725.1,197745,-10893,318,388,7460


There are 413 cases where the old patients had high blood pressure with their systolic number > 180

### 3. Data Cleaning:

The last column of the dataset is mispelled, so I rename the column:

In [8]:
colnames(mydata)[7] <- 'CIRCULATION'

In [9]:
colSums(is.na(mydata))

ACTIVITY        TIME          SL         EEG          BP          HR 
          0           0           0           0           0           0 
CIRCULATION 
          0

There's no NA values in this dataset.

### 4. Adaptation:

Create a new dataframe called bp that shows the mean blood pressure based on each activity, then rename the new dataframe:

In [10]:
bp = aggregate(BP~ACTIVITY, data = mydata, mean)
colnames(bp) <- c('ACTIVITY','Mean_BP')

Merge the original dataframe and the bp dataframe to have an additional column called Mean_BP:

In [11]:
mydata = merge(mydata, bp, by='ACTIVITY')
mydata$Mean_BP = round(mydata$Mean_BP,2)